# Import

In [1]:
import numpy as np
import tensorflow as tf

from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras import layers, models, losses

In [ ]:
!unzip Dataset.zip -d Dataset

# Data Generators

In [3]:
data_path = 'Dataset'
train_data_dir = data_path
img_height, img_width = 120, 120
batch_size = 64


train_datagen = ImageDataGenerator(rescale=1./255,
    validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle = True,
    color_mode = 'grayscale', #to grayscale
    subset='training') 

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle = True,
    color_mode = 'grayscale',
    subset='validation')

Found 30168 images belonging to 2 classes.
Found 7541 images belonging to 2 classes.


# Model (CNN Classifier) learning 

In [4]:
# Simple CNN two-class classifier
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', \
                        input_shape=(img_height, img_width, 1)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.Flatten())

model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(2))
model.summary()

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 118, 118, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 59, 59, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 57, 57, 32)        9248      
_________________________________________________________________
flatten (Flatten)            (None, 103968)            0         
_________________________________________________________________
dense (Dense)                (None, 100)               10396900  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 10,406,670
Trainable params: 10,406,670
Non-trainable params: 0
____________________________________________

In [5]:
# Learning
nb_epochs = 3
history = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/3
471/471 [==============================] - 475s 1s/step - loss: 0.1109 - accuracy: 0.9588 - val_loss: 0.0542 - val_accuracy: 0.9802
Epoch 2/3
471/471 [==============================] - 471s 1s/step - loss: 0.0381 - accuracy: 0.9888 - val_loss: 0.0240 - val_accuracy: 0.9915
Epoch 3/3
471/471 [==============================] - 471s 999ms/step - loss: 0.0245 - accuracy: 0.9927 - val_loss: 0.0133 - val_accuracy: 0.9963


# Model saving

In [6]:
model.save("./saved_models/my_model")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./saved_models/my_model/assets


In [7]:
!tar -czvf model.tar.gz saved_models

saved_models/
saved_models/my_model/
saved_models/my_model/assets/
saved_models/my_model/saved_model.pb
saved_models/my_model/variables/
saved_models/my_model/variables/variables.index
saved_models/my_model/variables/variables.data-00000-of-00001


# Testing and evaluation

In [8]:
test_generator = validation_generator

In [9]:
real_labels = []
predicted_labels = []
i=0
for batch in test_generator:
  if i >= test_generator.samples // batch_size:
    break
  i += 1
  pred_batch = model.predict_classes(batch[0])
  real_labels.extend(batch[1])
  predicted_labels.extend(pred_batch)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [10]:
real_labels = np.array(np.argmax(real_labels, axis=-1))
predicted_labels = np.array(predicted_labels)

In [11]:
print('F1 score on validation split: ',        f1_score(real_labels, predicted_labels))
print('Precision score on validation split: ', precision_score(real_labels, predicted_labels))
print('Recall score on validation split: ',    recall_score(real_labels, predicted_labels))
print('Accuracy score on validation split: ',  accuracy_score(real_labels, predicted_labels))

F1 score on validation split:  0.996264674493063
Precision score on validation split:  0.9975955116216938
Recall score on validation split:  0.9949373834265921
Accuracy score on validation split:  0.9962606837606838
